In [24]:
!apt-get install wget -y
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
!sudo apt-get update -y
!sudo apt-get install google-chrome-stable -y
!google-chrome --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
You might want to run 'apt --fix-broken install' to correct these.
The following packages have unmet dependencies:
 google-chrome-stable : Depends: fonts-liberation but it is not going to be installed
                        Depends: libgtk-3-0 (>= 3.9.10) but it is not going to be installed
                        Depends: libxkbcommon0 (>= 0.4.1) but it is not going to be installed
                        Depends: xdg-utils (>= 1.0.2) but it is not going to be installed
                        Recommends: libu2f-udev but it is not going to be installed
                        Recommends: libvulkan1 but it is not going to be installed
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google.list:1 and /etc/apt/sources.list.d/google.list:2
W: Target Packages (main/binary-all/Packages

Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1811 B]
Err:1 http://dl.google.com/linux/chrome/deb stable InRelease                   
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 78BD65473CB3BD13
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Reading package lists... Done       
W: GPG error: http://dl.google.com/linux/chrome/deb stable InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 78BD65473CB3BD13
E: The repository 'http://dl.google.com/linux/chrome/deb stable InRelease' is not signed.
N: Updating from such a repository can't be done securely, and is therefore disabled by default.
N: See apt-secur

In [6]:
!pip install selenium
!pip install webdriver_manager
!pip install tqdm

In [74]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time

from math import ceil
import time, json, os, argparse
from copy import deepcopy
from multiprocessing import Pool, Manager

from collections import OrderedDict
from itertools import repeat

In [43]:
def init_driver():

    options = webdriver.ChromeOptions()    
    options.add_argument('headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

    driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=options)

    driver.implicitly_wait(15)
    driver.get('https://papago.naver.com/')

    return driver

In [67]:
def papago_translate(corpus_number, translated, parts):
    corpus = parts[corpus_number]

    prev_result = result = ''
    kor2oth = []
    for idx, sentence in tqdm(enumerate(corpus)):
        if idx % 100 == 0:
            driver = init_driver()
            print('driver initialized')

        time_delay = 2
        while result == prev_result and time_delay < 10:
            if time_delay > 3:
                print(time_delay, result.split('\n'), sentence.split('\n'), sep='\n', end='\n\n')

            result = run_trans(sentence, driver, time_delay)

            time_delay += 1

        prev_result = result
            

        for en, tstd in zip(sentence.split('\n'), result.split('\n')):
            translated[en.strip()] = tstd.strip()

    if multiprocessor > 1:
        print(f'processor {corpus_number} completed')
    else:
        print('translation completed')
    return translated

In [ ]:
def kor2trans(translator,driver):

    input_box = driver.find_element_by_css_selector('#sourceEditArea textarea')        
    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 

    input_box.send_keys(sentence)

    driver.find_element_by_css_selector('#ddTargetLanguageButton').click()
    driver.find_element_by_xpath('//*[@id="ddTargetLanguage"]/div[2]/ul/li[2]').click()
    driver.find_element_by_css_selector('#btnTranslate').click()

    time.sleep(2)

    result = str(driver.find_element_by_css_selector("#txtTarget").text)

    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
    translator.append(result)
    
    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
    input_box.send_keys(sentence)
    
    return translator

In [ ]:
def trans2kor(translator,driver):

    input_box = driver.find_element_by_css_selector('#sourceEditArea textarea')        
    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 

    input_box.send_keys(sentence)

    driver.find_element_by_css_selector('#ddTargetLanguageButton').click()
    driver.find_element_by_xpath('//*[@id="ddTargetLanguage"]/div[2]/ul/li[2]').click()
    driver.find_element_by_css_selector('#btnTranslate').click()

    time.sleep(2)

    result = str(driver.find_element_by_css_selector("#txtTarget").text)

    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
    translator.append(result)
    
    return translator

In [75]:
def preprocessing_dataset(dataset, label_type):
  label = []
  for i in dataset[8]:
    if i == 'blind':
      label.append(100)
    else:
      label.append(label_type[i])
  out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_01_start':dataset[3],'entity_01_end':dataset[4]
                              ,'entity_02':dataset[5],'entity_02_start':dataset[6],'entity_02_end':dataset[7],'label':label,})
  return out_dataset

# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
  # load label_type, classes
  with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
  # load dataset
  dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
  # preprecessing dataset
  dataset = preprocessing_dataset(dataset, label_type)
  
  return dataset

In [76]:
data = load_data("/opt/ml/input/data/train/train.tsv")

data.head()

,sentence,entity_01,entity_01_start,entity_01_end,entity_02,entity_02_start,entity_02_end,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,8


In [70]:
parts = data.iloc[:]['sentence']
parts[:10]

0    영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...
1    선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...
2    유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...
3    용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...
4    람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...
5    이와 관련해 AFCD는 대학 소속 전문가와 세계동물보건기구(OIE)가 사람과 동물 ...
6    신형 K5 사전계약 돌풍에 대해 기아자동차 관계자는 "한눈에 각인될 정도의 강렬한 ...
7    2008년 이전 팀명칭은 나고야 그램퍼스 에이트였고 에이트는 나고야 시 표장에 있는...
8    SW융합교육원 조영주 담당교수는 “4차 산업혁명에 따라 SW역량은 전문가뿐만 아니라...
9    김동양 NH투자증권 연구원은 “삼성SDS의 3분기 매출액은 2조6604억원, 영업이...
Name: sentence, dtype: object

In [78]:
parts[0][data.iloc[0]['entity_01_start']:data.iloc[0]['entity_01_end']+1]

'랜드로버'

In [79]:
parts[0][data.iloc[0]['entity_02_start']:data.iloc[0]['entity_02_end']+1]

'자동차'

In [ ]:
corpus = load_text(english_dir)
n = len(corpus)
print(f'You have left {n}') 

n = len(corpus)
parts = [deepcopy(corpus[ceil(n/multiprocessor)*i:ceil(n/multiprocessor)*(i+1)]) for i in range(8)]
print(len(parts))

In [71]:
pool = Pool(8)
manager = Manager()
translated = manager.dict()
pool.starmap(papago_translate, zip(range(8), repeat(translated), parts))
pool.close()
pool.join()



0it [00:00, ?it/s]

====== WebDriver manager ======
0it [00:00, ?it/s]

0it [00:00, ?it/s]====== WebDriver manager ======


0it [00:00, ?it/s]====== WebDriver manager ======


0it [00:00, ?it/s]====== WebDriver manager ======


====== WebDriver manager ======
0it [00:00, ?it/s]

0it [00:00, ?it/s]====== WebDriver manager ======


0it [00:00, ?it/s]====== WebDriver manager ======


Current google-chrome version is 90.0.4430
====== WebDriver manager ======
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Current google-chrome version is 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Cur

driver initialized


0it [00:02, ?it/s]


driver initialized


0it [00:02, ?it/s]


...

0it [00:03, ?it/s]


0it [00:03, ?it/s]


driver initialized


0it [00:03, ?it/s]


NameError: name 'run_trans' is not defined

In [58]:


driver = init_driver()
print('driver initialized')
translator = []
for sentence in tqdm(data['sentence']):
# sentence = '무하의 애국심은 히틀러의 독일 정부가 그를의 적으로 간주한 명부에 포함할 만큼 유명했다.'
# 무하의 애국심은 매우 유명해서 히틀러의 독일 정부는 그를 적으로 간주되는 명단에 포함시켰다.

    input_box = driver.find_element_by_css_selector('#sourceEditArea textarea')        
    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 

    input_box.send_keys(sentence)

    driver.find_element_by_css_selector('#ddTargetLanguageButton').click()
    driver.find_element_by_xpath('//*[@id="ddTargetLanguage"]/div[2]/ul/li[2]').click()
    driver.find_element_by_css_selector('#btnTranslate').click()

    time.sleep(2)

    result = str(driver.find_element_by_css_selector("#txtTarget").text)

    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
    translator.append(result)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/opt/ml/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
  0%|          | 0/9000 [00:00<?, ?it/s]

driver initialized


  1%|          | 58/9000 [02:45<7:05:56,  2.86s/it]


KeyboardInterrupt: 

In [40]:
driver = init_driver()
print('driver initialized')
result_tans = []
# sentence = "Muha's patriotism was so famous that Hitler's German government included him on the list of considered enemies."
for sentence in translaor:
    input_box = driver.find_element_by_css_selector('#sourceEditArea textarea')        
    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 

    input_box.send_keys(sentence)

    driver.find_element_by_css_selector('#ddTargetLanguageButton').click()
    driver.find_element_by_xpath('//*[@id="ddTargetLanguage"]/div[2]/ul/li[1]').click()
    driver.find_element_by_css_selector('#btnTranslate').click()

    time.sleep(2)

    result = str(driver.find_element_by_css_selector("#txtTarget").text)

    input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
    result_trans.append(result)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/opt/ml/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


driver initialized
무하의 애국심은 매우 유명해서 히틀러의 독일 정부는 그를 적으로 간주되는 명단에 포함시켰다.


sentence     선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...
entity_01                                                  민주당
entity_02                                                  27석
label                                                        0
Name: 1, dtype: object

In [56]:
for i in range(len(data)):
    data.iloc[i]['sentence'] = result_trans[i]

NameError: name 'result_trans' is not defined

In [ ]:


class Translator(object):
    def __init__(self, train_dir, papago_dir, multiprocessor=1):
        self.train_dir = train_dir
        self.papago_dir = papago_dir
        self.multiprocessor = multiprocessor

        manager = Manager()
        self.translated = manager.dict()
        self.parts = [deepcopy(corpus[ceil(self.n/multiprocessor)*i:ceil(self.n/multiprocessor)*(i+1)]) for i in range(self.multiprocessor)]

    def en2kr(self, sentence, driver, time_delay):

        input_box = driver.find_element_by_css_selector('#sourceEditArea textarea')
        
        input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
        
        input_box.send_keys(sentence)
        
        driver.find_element_by_css_selector('#ddTargetLanguageButton').click()
        driver.find_element_by_xpath_selector('//*[@id="ddTargetLanguage"]/div[2]/ul/li[3]').click()
        driver.find_element_by_css_selector('#btnTranslate').click()

        time.sleep(time_delay)
        
        result = str(driver.find_element_by_css_selector("#txtTarget").text)

        input_box.clear(); input_box.clear(); input_box.clear(); input_box.clear(); 
        return result

    def translate(self):
        pool = Pool(self.multiprocessor)
        pool.starmap(self._translate, zip(range(self.multiprocessor), repeat(self.translated)))
        pool.close()
        pool.join()

        self.save_json(self.korean_dir, self.translated._getvalue())
        print('file translation completed')

    def _translate(self, corpus_number, translated):
        corpus = self.parts[corpus_number]

        prev_result = result = ''

        for idx, sentence in tqdm(enumerate(corpus)):
            if idx % 100 == 0:
                driver = self.init_driver()
                print('driver initialized')
            
            time_delay = 2
            while result == prev_result and time_delay < 10:
                if time_delay > 3:
                    print(time_delay, result.split('\n'), sentence.split('\n'), sep='\n', end='\n\n')
                
                result = self.en2kr(sentence, driver, time_delay)
                time_delay += 1
            
            prev_result = result

            for en, tstd in zip(sentence.split('\n'), result.split('\n')):
                translated[en.strip()] = tstd.strip()
            
        if self.multiprocessor > 1:
            print(f'processor {corpus_number} completed')
        else:
            print('translation completed')
        return translated
    
    def load_json(self, json_dir):
        with open(json_dir, encoding='UTF8') as f:
            json_file = json.load(f)
        print(f'{json_dir} loaded')
        return json_file
    
    def save_json(self, save_dir, result_dict):
        with open(save_dir,'w', encoding='UTF8') as f:
            json.dump(result_dict, f, ensure_ascii=False)
    
    def load_text(self, text_dir):
        with open(text_dir, encoding='UTF8') as f:
            text_file = f.readlines()
        print(f'{text_dir} loaded')
        return text_file

    def init_driver(self):

        options = webdriver.ChromeOptions()    
        options.add_argument('headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')

        options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

        driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), chrome_options=options)

        driver.implicitly_wait(15)
        driver.get('https://papago.naver.com/')

        return driver


if __name__ == "__main__":
    print(f"Start")
    start_time = time.time()
    parser = argparse.ArgumentParser(description='Before get started you need read README')

    parser.add_argument('--train_dir', type=str, help='directory of train tsv file',
                        default='/opt/ml/input/data/train/train.tsv')

    parser.add_argument('--result_dir', type=str, help='directory of translated tsv to save',
                        default='/opt/ml/input/data/train/papago.tsv')
    
    parser.add_argument('--multiprocessor', type=int, help='multiprocessor number you want to use',
                        default=int(1))                    

#     parser.add_argument('--path', type=str, help='type chromedriver\'s abs dir',
#                         default='chromedriver')                    


    args = parser.parse_args()
    
    translator = Translator(args.english_dir, args.kr_json_dir, args.multiprocessor)

    translator.translate()
    print(f"Complete translate.")
    print(f'Time consumption : {time.time() - start_time}')